# Projeto 1 - Ciência dos Dados

Nome: Pedro Herique Britto Aragão Andrade

Nome: Thiago Shiguero Kawahara

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [169]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [170]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\thiag\Projeto1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [171]:
filename = 'Pringles.xlsx'

In [172]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,eu 🤝 comer pringles com danette\n\n combinaç...,1
1,"@englantinez pipoca, bolinha de amendoim ou ba...",1
2,chorando e comendo pringles,0
3,leticia podendo me viciar em tanta coisa me vi...,1
4,vontade de uma pringles ughhhhh,0


In [173]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,o cara tem um pote de pringles entre as perna ...,0
1,@ellixquintx a pringles de cebola maior que ca...,0
2,se eu fosse líder eu n ia falar com a pessoa q...,1
3,vou fazer meu vício favorito dos últimos tempo...,1
4,ai to muito puta namoral ja era a minha coca c...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


Pringles é um aperitivo no formato batata chips e vendido em um tubo cilindrico.

Relevante: avaliacoes negativas do produto ou que tivessem informacoes interessantes para uma possivel parceria ou novas ideias de marketing.

Nao relevante: todos que nao fossem relevantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [174]:
# funcao para limpar o texto de pontuacoes

import re 

def cleanup(text):
    punctuation = '[!-.:?;¿]' 
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [175]:
# separando a tabela de treinamento em relevante e irrelevante

train_i = train.loc[train.Classificação==0,:]

train_r = train.loc[train.Classificação==1,:]

In [176]:
# limpando (cleanup()), juntando todas as palavras da tabela treinamento (.join()) e individualizando cada palavra (.split())


# para tabela total
l_t=[]
for i in train.Treinamento:
    l_t.append(cleanup(i))
l_t = " ".join(l_t).split()


# para tabela somente relevante
l_r=[]
for i in train_r.Treinamento:
    l_r.append(cleanup(i))   
l_r = " ".join(l_r).split()


# para tabela irrelevante 
l_i=[]
for i in train_i.Treinamento:
    l_i.append(cleanup(i))    
l_i = " ".join(l_i).split()

In [177]:
tab_total = pd.Series(l_t)
tab_total_a = tab_total.value_counts()
tab_total_r = tab_total.value_counts(True)

tab_rel = pd.Series(l_r)
tab_rel_a = tab_rel.value_counts()
tab_rel_r = tab_rel.value_counts(True)

tab_irr = pd.Series(l_i)
tab_irr_a = tab_irr.value_counts()
tab_irr_r = tab_irr.value_counts(True)

In [178]:
# calculo da probabilidade de ser relevante e da probabilidade de ser irrelevante

probR = sum(tab_rel_a)/sum(tab_total_a)
probI = sum(tab_irr_a)/sum(tab_total_a)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [179]:
# limpando e separando cada frase da tabela teste

l_teste=[]
for i in test.Teste:
    c = cleanup(i)
    l_teste.append(c.split())

In [182]:
# cridando um DataFrame para o maquina preencher

data = {'Frase':  [],
        'Classificação': [],
    }

novo = pd.DataFrame(data, columns = ['Frase','Classificação'])

In [188]:
# Legenda: T == Tweet

contador = 0
for i in l_teste:
    probTDadoR = 1
    for j in i:
        if j in tab_rel_a:
            probTDadoR*=(tab_rel_r[j])
            
    probTDadoI = 1
    for j in i:
        if j in tab_irr_a:
            probTDadoI*=(tab_irr_r[j])
            
    probRdadoT = probTDadoR*probR
    
    probIdadoT = probTDadoI*probI

    if probRdadoT>probIdadoT:
        
        novo.loc[contador,"Frase"]= " ".join(i)
        novo.loc[novo.Frase==" ".join(i),"Classificação"] = "Relevante"
    
    elif probRdadoT<probIdadoT:
        
        novo.loc[contador,"Frase"]=" ".join(i)
        novo.loc[novo.Frase==" ".join(i),"Classificação"] = "Irrelevante"
    
    contador+=1

### Para comparar

In [192]:
test.groupby(['Classificação']).count()

,Teste
Classificação,
0,142
1,58


In [190]:
novo.groupby(['Classificação']).count()

,Frase
Classificação,
Irrelevante,72
Relevante,128


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**